In [1]:
from ppsim import Simulation, StatePlotter, time_trials
from dataclasses import dataclass
import dataclasses
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import pickle
%matplotlib widget
import ipywidgets as widgets

# Simplest protocols for the majority problem

The majority problem has a simple 4 state solution, which was analyzed [here](https://arxiv.org/abs/1202.1083) and [here](https://arxiv.org/abs/1404.7671). The rule is always correct, by maintaining the invariant #A - #B.

In [2]:
exact_majority = {
    ('A', 'B'): ('a', 'b'),
    ('A', 'b'): ('A', 'a'),
    ('B', 'a'): ('B', 'b')
}

In the worst case, where the initial gap (#A - #B) is constant, this takes $\Theta(n \log n)$ time to reach the stable correct output configuration.

In [3]:
n = 10 ** 5
init_config = {'A': n // 2 + 1, 'B': n // 2}
sim = Simulation(init_config, exact_majority, transition_order='symmetric')
sim.run()
sim.history.plot()
plt.title('4 state majority protocol')
plt.xscale('symlog')
plt.yscale('symlog')
plt.xlim(0, sim.times[-1])
plt.ylim(0, n)

 Time: 490242.000


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0, 100000.0)

In the case of a tie, the 4 state protocol does not have well-defined behavior. But by adding two more states, we can correct detect ties as well.

In [4]:
# states are A, B, T, a, b, t
def exact_majority_ties(x, y):
    # Cancellation
    if x == 'A' and y == 'B':
        return ('T', 'T')
    # Active A / B eliminate T
    if x in ['A', 'B'] and y == 'T':
        return (x, x.lower())
    # Active converts passive
    if x.isupper() and y.islower():
        return (x, x.lower())

n = 10 ** 5
sim = Simulation({'A': n // 2, 'B': n // 2}, exact_majority_ties, transition_order='symmetric')
print(sim.reactions)
sim.run()
sim.history.plot()
plt.title('6 state majority protocol detecting ties')
plt.xscale('symlog')
plt.yscale('symlog')
plt.xlim(0, sim.times[-1])
plt.ylim(0, n)

A, B  -->  T, T
A, T  -->  A, a
B, T  -->  B, b
B, a  -->  B, b
T, a  -->  T, t
A, b  -->  A, a
T, b  -->  T, t
A, t  -->  A, a
B, t  -->  B, b
 Time: 244033.000


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0, 100000.0)

Another simple example is the 3-state approximate majority protocol, which was analyzed [here](http://www.cs.yale.edu/homes/aspnes/papers/approximate-majority-journal.pdf) and [here](https://www.cs.ubc.ca/~condon/papers/approx-maj-journal.pdf).

In [5]:
a, b, u = 'A', 'B', 'U'
approximate_majority = {
    (a,b): (u,u),
    (a,u): (a,a),
    (b,u): (b,b)
}
n = 10 ** 9
init_config = {a: n // 2 * 0.5001, b: n // 2 * 0.4999}
sim = Simulation(init_config, approximate_majority)
sim.run(recording_step=0.1)
sim.history.plot()
plt.title('3 state approximate majority protocol')

 Time: 51.400


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, '3 state approximate majority protocol')

It was shown to stabilize in only $O(\log n)$ time to a consensus configuration.

In [ ]:
ns = [int(n) for n in np.geomspace(10, 10 ** 8, 20)]
def initial_condition(n):
    return {'A': n // 2, 'B': n // 2}
df = time_trials(approximate_majority, ns, initial_condition, num_trials=100, max_wallclock_time = 30, transition_order='symmetric')
fig, ax = plt.subplots()
ax = sns.lineplot(x='n', y='time', data=df)
ax.set_title('Average stabilization time of approximate majority')
ax.set_xscale('log')

This consensus will only be correct with high probability, however, and requires the initial gap to be $\Omega(\sqrt{n \log n})$. We can see that when the gap is close to 0, it is performing essentially a random walk, which is why a sufficiently large initial gap is necessary to ensure the initial majority stays ahead.

In [ ]:
sim.reset({a: n // 2 + 1, b: n // 2 - 1})
sim.run(4, recording_step = 0.01)
fig, ax = plt.subplots()
ax.set_title('Count of A - count of B')
ax.set_yscale('symlog')
(sim.history['A'] - sim.history['B']).plot()

# Bias Averaging Framework for $O(\log n)$ state protocols

We view the initial states `A` and `B` as having `bias = +1` and `bias = -1` respectively. We then maintain the invariant that all interactions preserve the total bias.
To bound the total number of states to $O(\log n)$, the only allowable values for `bias` will be $\pm 1, \pm\frac{1}{2}, \pm\frac{1}{4}, \ldots, \pm\frac{1}{2^L}$ where $L = \lceil \log_2(n) \rceil$.
We describe the state of the agent with two fields `opinion`$=\pm 1$ and `exponent`$=0,-1, \ldots, -L$, so `bias = opinion * (2 ** exponent)`.

In [ ]:
from fractions import Fraction

@dataclass(unsafe_hash=True)
class Agent:
    opinion: int = 0
    exponent: int = 0
        
    @property
    def bias(self):
        return self.opinion * 2 ** self.exponent
    
    @bias.setter
    def bias(self, value):
        if value == 0:
            self.opinion = self.exponent = 0
        else:
            self.opinion = int(np.sign(value))
            exponent = np.log2(abs(value))
            if exponent.is_integer():
                self.exponent = int(exponent)
            else:
                raise ValueError(f'bias = {value} must an integer power of 2')
    
    def __str__(self):
        if self.bias == 0:
            return '0'
        s = ''
        if self.bias > 0:
            s += '+'
        if abs(self.bias) > 1/100:
            s += str(Fraction(self.bias))
        else:
            if self.bias < 0:
                s += '-'
            s += '1/2^' + str(abs(self.exponent))
        return s
    
def init_agents(a, b):
    return {Agent(opinion = 1): a, Agent(opinion = -1): b}

The cancel / split reactions maintain the invariant sum of agent biases.

In [ ]:
def cancel_split(a: Agent, b: Agent, L: int):
     # cancel reaction
    if a.bias == -b.bias:
        a.opinion = b.opinion = 0
        a.exponent = b.exponent = 0
    
    # split reaction
    if a.bias == 0 and abs(b.bias) > 2 ** (-L):
        a.opinion = b.opinion
        a.exponent = b.exponent = b.exponent - 1
    
    if b.bias == 0 and abs(a.bias) > 2 ** (-L):
        b.opinion = a.opinion
        b.exponent = a.exponent = a.exponent - 1

print(Simulation(init_agents(1, 1), cancel_split, L = 4).reactions)

By themselves, however, these rules do not solve majority.

In [ ]:
n = 10 ** 6
sim = Simulation(init_agents(n // 2 + 1, n // 2), cancel_split, L=int(np.log2(n)))
sp = StatePlotter()
sim.add_snapshot(sp)
sp.ax.set_yscale('symlog')

In [ ]:
sim.run(recording_step=0.1)
sim.snapshot_slider()

There are a few additional transitions that will also preserve the bias.

In [ ]:
from itertools import product

def bias_average(a, b, L):
    a, b = dataclasses.replace(a), dataclasses.replace(b)
    
    # all allowable bias values
    biases = [0] + [2 ** i for i in range(-L,1)] + [-2 ** i for i in range(-L, 1)]
    # all pairs of bias values that preserve the sum
    legal_outputs = [(x,y) for (x,y) in product(biases, biases) if x + y == a.bias + b.bias]
    # choose the pair of bias values which are closest together
    a.bias, b.bias = legal_outputs[np.argmin(np.array([abs(x-y) for (x,y) in legal_outputs]))]
    
    return a, b

print(Simulation(init_agents(1, 1), bias_average, L = 4).reactions)

But just these transitions do not speed up the protocol or remove the probability of error.

In [ ]:
n = 10 ** 6
sim = Simulation(init_agents(n // 2 + 1, n // 2), bias_average, L=int(np.log2(n)))
sp = StatePlotter()
sim.add_snapshot(sp)
sp.ax.set_yscale('symlog')

In [ ]:
sim.run(recording_step=0.1)
sim.snapshot_slider()

Here was an example simulation run where some minority agents were never eliminated:

In [ ]:
sim = pickle.load( open( "majority_simulations/bias_average.p", "rb" ) )
sim.snapshot_slider()

# Adding Synchronization

The unbiased agents will now have a field `hour`, and will wait until `hour = i` before doing a split down to `exponent = -i`.
They will synchronize their `hour` with separate clock agents who are keeping a timer through a field `minute`, where `hour = minute // m` for a parameter `m` which gives the number of minutes per hour.

In [ ]:
@dataclass(unsafe_hash=True)
class MajorityAgent(Agent):
    role: str = 'main'
    _hour: int = 0
    minute: int = 0
    finished: bool = False
    m: int = 5
        
    @property
    def hour(self):
        if self.role == 'clock':
            return self.minute // self.m
        else:
            return self._hour
        
    @hour.setter
    def hour(self, value):
        if self.role == 'main':
            self._hour = value
        # can't change hour for a clock agent
    
    def __str__(self):
        if self.bias != 0:
            return super().__str__()
        if self.role == 'clock':
            return 'c' + str(self.minute)
        else:
            return 'u' + str(self.hour)
    
def init_majority_agents(a, b, m):
    return {MajorityAgent(opinion = 1, m = m): a, MajorityAgent(opinion = -1, m = m): b}

# custom function to build plots that visualize the 3 populations of clock, unbiased, and biased agents
def make_plots(sim):
    plt.ioff()
    clock_plot = StatePlotter(lambda a: a.minute if a.role == 'clock' else None, update_time = 1)
    sim.add_snapshot(clock_plot)
    clock_plot.ax.set_xlabel('clock minute')
    clock_plot.ax.axes.xaxis.set_ticklabels([])
    unbiased_plot = StatePlotter(lambda a: a.hour if a.role == 'main' and a.bias == 0 else None, update_time = 1)
    sim.add_snapshot(unbiased_plot)
    unbiased_plot.ax.set_xlabel('unbiased hour')
    biased_plot = StatePlotter(lambda a: str(a) if a.bias != 0 else None, update_time = 1)
    sim.add_snapshot(biased_plot)
    for snap in sim.snapshots:
        snap.ax.set_yscale('symlog')
        snap.fig.tight_layout()
    plt.ion()
    sim.layout = widgets.GridspecLayout(6,2, height='700px', pane_heights=[4,7,1], grid_gap='5px')
    sim.layout[0:2,0] = clock_plot.fig.canvas
    sim.layout[0:2,1] = unbiased_plot.fig.canvas
    sim.layout[2:5,:] = biased_plot.fig.canvas
    sim.layout[5,:] = sim.snapshot_slider()
    display(sim.layout)

The clock agents will count for an additional `L` minutes after the last hour ($O(\log n)$ time). Then they will send a signal `finished = True` that makes all agents stop (and move on to a later phase of the algorithm).

In [ ]:
def majority(a, b, L):
    a.finished = b.finished = a.finished or b.finished
    if a.finished:
        a.minute = b.minute = 0
        a.hour = b.hour = 0
    else:
        if a.role == b.role == 'main':
            # cancel reaction
            if a.bias == -b.bias != 0:
                a.opinion = b.opinion = 0
                a.hour = b.hour = abs(a.exponent)
                a.exponent = b.exponent = 0
                # half the agents from first split become clock
                if a.hour == 0:
                    a.role = 'clock'

            # split reaction
            if a.bias == 0 and b.bias != 0 and a.hour > abs(b.exponent):
                a.opinion = b.opinion
                a.exponent = b.exponent = b.exponent - 1
                a.hour = b.hour = 0

            if b.bias == 0 and a.bias != 0 and b.hour > abs(a.exponent) :
                b.opinion = a.opinion
                b.exponent = a.exponent = a.exponent - 1
                a.hour = b.hour = 0

        # unbiased agents propagate max hour
        if a.bias == b.bias == 0:
            a.hour = b.hour = min(max(a.hour, b.hour), L)

        # clock minute uses new fixed resolution phase clock
        if a.role == b.role == 'clock':
            # drip reaction
            if a.minute == b.minute:
                a.minute += 1
#               Wait an additional L minutes after hour L before finishing
                if a.minute == a.m * L + L:
                    a.finished = True
            # epidemic reaction
            else:
                a.minute = b.minute = max(a.minute, b.minute)

If we set the number of minutes per hour `m` to be $O(\log n)$ then with high probability the entire population will stay synchronized at the same hour. In this case, we have an $O(\log^2 n)$ time majority algorithm, essentially the same as the standard 'canceling and doubling' protocols.

In [ ]:
n = 10 ** 6
sim = Simulation(init_majority_agents(n // 2 + 1, n // 2, m = int(np.log(n))), majority, L=int(np.log2(n)))
make_plots(sim)

In [ ]:
sim.run()
sim.layout[5,:] = sim.snapshot_slider()

To make the protocol take only $O(\log n)$ time, we set the parameter `m` to be constant. In the case of a tie, we will end up with every biased agent reaching the minimum value `exponent = -L`. Choosing $L = \lceil \log_2(n) \rceil$ ensures that this can only happen in the case of a tie. Thus we can check if all exponents are `-L` after this phase finishes to stably detect a tie.

In [ ]:
n = 10 ** 7
sim = Simulation(init_majority_agents(n // 2, n // 2, m = 3), majority, L=int(np.log2(n)))
make_plots(sim)

In [ ]:
sim.run()
sim.layout[5,:] = sim.snapshot_slider()

In the more general case, we will not eliminate all minority agents. What will be true, with high probability, is that a vast majority of agents will finish with the majority opinion, in a range of 3 consecutive exponents.

In [ ]:
n = 10 ** 7
sim = Simulation(init_majority_agents(n // 2 + int(n ** 0.5), n // 2 - int(n ** 0.5), m = 3), majority, L=int(np.log2(n)))
sim.run()
make_plots(sim)

In [ ]:
sim.run()
sim.layout[5,:] = sim.snapshot_slider()

In [ ]:
## For a larger value of n, a simulation was ran and then pickled
# n = 10 ** 10
# sim = Simulation(init_majority_agents(n // 2 + int(n ** 0.5), n // 2 - int(n ** 0.5), m = 3), majority, L=int(np.log2(n)))
# sim.run()
# pickle.dump(sim, open( "majority_simulations/majority.p", "wb" ) )

# We can now load this simulation
sim = pickle.load( open( "majority_simulations/majority.p", "rb" ) )
make_plots(sim)

# Clock Protocol

Looking more closely at the rule of the `clock` agents, we can see the key important feature of the `minute` distribution is that the front tail decays doubly-exponentially, while the back tail decays exponentially. This ends up ensuring that when a majority of agents are in `hour = h`, the fraction of agents with `hour > h` can be made to be a fraction that is arbitrarily small by tuning the parameter `m`.

In [ ]:
def clock(a, b, m):
    if a == b < m:
        return a + 1, b
    else:
        return max(a, b), max(a, b)

In [ ]:
n = 10 ** 9
sim = Simulation({0: n}, clock, m = 30)
sp = StatePlotter()
sim.add_snapshot(sp)
sp.ax.set_yscale('symlog')

In [ ]:
sim.run(recording_step=0.1)
sim.snapshot_slider()

Notice also that this clock rule is extremely similar to the power-of-two-choices phase clock. In fact, the distribution of the clock ends up being essentially the same.

In [ ]:
def two_choices_clock(a, b, m):
    if min(a, b) < m:
        return min(a, b) + 1, max(a, b)

In [ ]:
n = 10 ** 9
sim = Simulation({0: n}, two_choices_clock, m = 30)
sp = StatePlotter()
sim.add_snapshot(sp)
sp.ax.set_yscale('symlog')

In [ ]:
sim.run(recording_step=0.1)
sim.snapshot_slider()